In [1]:
import datetime as dt
import glob
import json
import pandas as pd
import pyodbc

from yahoo_fin.stock_info import get_data


### Extract and Preprocess Data

In [4]:
# Dictionary mapping ticker to corresponding metal name
ticker_dict = {'GC=F':'gold', 'HG=F':'copper', 'SI=F':'silver', 'PL=F':'platinum', 'PA=F':'palladium'}

In [8]:
# Return dataframe contain data extracted from Yahoo Finance for specified ticker from 01/01/2000 to current date
def extract_data(ticker, start_date="01/01/2000", end_date=dt.date.today()):
    data = get_data(ticker, start_date=start_date, end_date=end_date, index_as_date = False, interval="1d")
    return pd.DataFrame(data)

# Remove null, duplicates, typecast and unneeded columns. Rename columns
def preprocess(df):
    df.dropna(inplace=True)
    df.drop_duplicates(subset=['date'], inplace=True)
    df['ticker'] = metals
    df['volume'] = df['volume'].astype('int')
    df.drop(['adjclose'], inplace=True, axis=1)
    df.rename(columns={'ticker':'name', 'open':'opening_price', 'high':'highest_price', 'low':'lowest_price', 'close':'closing_price', 'volume':'transaction_count'}, inplace=True)
    return df

In [9]:
# For every ticker key in ticker_dict create a pandas dataframe value named after the metal
# Ex. copper, gold, etc
df_dict = {}
for tickers in ticker_dict.keys():
    df_dict[ticker_dict[tickers]] = extract_data(ticker=tickers)
    

In [11]:
# For every key values pair in df_dict perform preprocess function and data as .csv file
for metals in df_dict.keys():
    processed_df = preprocess(df_dict[metals])
    processed_df.to_csv(f'future_data/{metals}.csv', index=False)


### Database Table Creation

In [12]:
# Collect path of all relevant csv file in one place
future_folder = glob.glob('future_data/*.csv')

# Load in secret file containing database connection credential. Connect to the database
secret = json.load(open('secret.json'))
conn = pyodbc.connect(DRIVER='{SQL Server}', 
                      SERVER=secret['server'], 
                      DATABASE=secret['database'], 
                      UID=secret['username'], 
                      PWD=secret['password'],
                      Trusted_connection='no')
cursor = conn.cursor()

In [13]:
# Functions to create table if didn't exist previously
def create_table(metal_name):
  cursor.execute(
  f"""
  ALTER DATABASE Metal_Future SET RECURSIVE_TRIGGERS OFF

  IF OBJECT_ID(N'Metal_Future.dbo.{metal_name}', N'U') IS NULL
    CREATE TABLE Metal_Future.dbo.{metal_name} (
    id INT IDENTITY(1,1) PRIMARY KEY,
    [date] DATE,
    name VARCHAR(20),
    opening_price FLOAT,
    closing_price FLOAT,
    highest_price FLOAT,
    lowest_price FLOAT,
    transaction_count INT)
    """)
  conn.commit()

# Function to create a lookup table mapping ticker and metal
def create_lookup_table():
    cursor.execute(
    """
    IF OBJECT_ID(N'Metal_Future.dbo.ticker_name', N'U') IS NULL
        CREATE TABLE Metal_Future.dbo.ticker_name (
        ticker VARCHAR(8) PRIMARY KEY,
        name VARCHAR(20))
    """)
    conn.commit()

# Function to insert ticker name and metal name in lookup table
def insert_lookup_table(ticker_dict):
    for keys in ticker_dict.keys():
        cursor.execute(
            f"""
            INSERT INTO Metal_Future.dbo.ticker_name (ticker, name)
            VALUES (?, ?)
            """,
            keys, ticker_dict[keys]
            )
        conn.commit()

# Check a row to see if highest col contains the highest value. If not, replace highest col value with the higher value between opening and closing col. 
def validate_highest(highest, opening, closing):
    if highest < opening and opening >= closing:
        return opening
    elif highest < closing and opening < closing:
        return closing
    else:
        return highest
    
# Check a row to see if highest col contains the lowest value. If not, replace lowest col value with the lower value between opening and closing col. 
def validate_lowest(lowest, opening, closing):
    if lowest > opening and opening <= closing:
        return opening
    elif lowest > closing and opening > closing:
        return closing
    else:
        return lowest

# Function to insert all rows of specified dataframe into a table. All validation steps are perform before insert.
def validate_and_insert(df, metal_name):
       for row in df.itertuples():
              cursor.execute(f"""
                     INSERT INTO Metal_Future.dbo.{metal_name} (date, name, opening_price, closing_price, highest_price, lowest_price, transaction_count)
                     VALUES (?, ?, ?, ?, ?, ?, ?)
                     """,
                     row.date, row.name, row.opening_price, row.closing_price, validate_highest(row.highest_price, row.opening_price, row.closing_price), validate_lowest(row.lowest_price, row.opening_price, row.closing_price), row.transaction_count
                     )
       conn.commit()


In [14]:
# Create and insert lookup table in database
create_lookup_table()
insert_lookup_table(ticker_dict)

In [15]:
# For each csv within future_folder, create a table and insert data entries into database
for csv in future_folder:
    df = pd.read_csv(csv)
    metal_name = csv.split('\\')[1].split('.')[0]
    create_table(metal_name)
    validate_and_insert(df, metal_name)